# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\James\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\James\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\James\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///distab.db')
df = pd.read_sql("SELECT * FROM DisasterTable", engine)

X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    
    # clean punctuation
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
        

    tokens = word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(word).lower() for word in tokens]
    tokens = [word for word in tokens if not word in stop_words]
    
    return tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [6]:
tokenize(X_train.loc[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [13]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize)),
    ('svd', TruncatedSVD(n_components=100)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    #('clf', MultiOutputClassifier(GradientBoostingClassifier()))
    #('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
predictions = pd.DataFrame(pred, columns=y.columns)
predictions.head()

In [ ]:
for col in predictions.columns:
    print("=========================\n")
    print(f'{col}\n')
    print(classification_report(predictions[col], y_test[col]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__ngram_range': [(1,1), (1,2), (1,3)],
             'vect__max_df': [0.7, 0.8, 1.0],
             'vect__max_features': [None, 1500, 3000],
             'svd__n_components': [300, 350, 400],
             'clf__n_estimators': [100, 300, 500],
             'clf__learning_rate', [0.1, 0.01, 0.001]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='log_loss', n_jobs=4, cv=5, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.